<center>
    <h1 style="color:deeppink">Just between us SQL Friends...</h1>
    <h2 style="color:deeppink">RuPaul's Drag Race: Visualized</h2>
    <img src="media/header.gif" height="100px">
</center>

<h2 style="color:deeppink">Import Libraries & Connect SQLite Database</h2>

In [1]:
import pandas as pd
import sqlite3
from pandas.io.json import json_normalize

drag_race_data_base = sqlite3.connect("drag_race_data.db")

<h2 style="color:deeppink">Define Constants, Global Variables, and API endpoints</h2>

In [11]:
DRAG_RACE_API_URL = "http://www.nokeynoshade.party/api/"

#API Endpoints
SEASONS = "seasons"
QUEENS = "queens/all"
SEASON_BY_NUMBER = "seasons/{}"

<h2 style="color:deeppink">Define Functions</h2>

In [12]:
def get_data(endpoint, SEASON = None):
    data = pd.read_json(DRAG_RACE_API_URL + endpoint.format(SEASON))
    return data


def get_seasons():
    df = pd.DataFrame()
    temp_df = pd.DataFrame()
    for i in range(len(season_id_df)):
        temp_df = get_data("seasons/{}", i + 1)
        temp_df = json_normalize(temp_df['queens'])
        temp_df['season_id'] = i + 1
        df = pd.concat([df, temp_df], sort=False)
        i += 1
    return df


def csv_export(df, file_name):
    df.to_csv(file_name, index=True)

<h2 style="color:deeppink">Define SQL Queries</h2>

In [13]:
SELECT_QUEENS = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
"""

all_queens_query = SELECT_QUEENS + """
    ORDER BY season_number, place DESC
"""


all_stars_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE season_number LIKE "%A%"
    ORDER BY season_number, place DESC
"""


main_seasons_query = SELECT_QUEENS + """
    WHERE season_number NOT LIKE "A%" 
    ORDER BY season_number, place DESC
"""


main_season_winners_query = SELECT_QUEENS + """
    WHERE winner IS True AND season_number NOT LIKE "A%" AND place = 1
"""


all_stars_winners_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE season_number LIKE "A%" AND place = 1
"""


miss_congeniality_query = """
    SELECT * FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
""" + """
    INNER JOIN queens ON seasons.id = queens.id
    WHERE missCongeniality IS True
    ORDER BY season_id"""

<h2 style="color:deeppink">Define values used for cleanup</h2>

In [5]:
queens_drop_cols = ['queens', 'name']

<h2 style="color:deeppink">Create raw dataframes from API endpoints</h2>

In [6]:
raw_seasons_df = get_data(SEASONS)
raw_queens_df = get_data(QUEENS)

# create season_id_df to look up season numbers by id
season_id_df = pd.DataFrame({"season_id": raw_seasons_df['id'], "season_number": raw_seasons_df['seasonNumber']})

<h2 style="color:deeppink">Clean and organize the data</h2>

<h2 style="color:deeppink">Create SQL Tables</h2>

In [7]:
# queens table
raw_queens_df.to_sql('queens', drag_race_data_base, if_exists='replace')

# seasons table
seasons_df = get_seasons().set_index('id').drop(queens_drop_cols, axis=1)
seasons_df.to_sql('seasons', drag_race_data_base, if_exists = 'replace')

# season_ids table
season_id_df.to_sql('season_ids', drag_race_data_base, if_exists='replace')

<h2 style="color:deeppink">Create DataFrames from the SQL Tables</h2>

In [8]:
#FIX THE SORTING FOR main_seasons... needs to be sorted by season ascending, then place descending. 
# the dtype change is what is causing the trouble


#all queens, ranked
main_seasons_df = pd.read_sql(main_seasons_query, drag_race_data_base).astype({'season_number':'int32'}).sort_values(by=['season_number']).set_index('id')
all_stars_df = pd.read_sql(all_stars_query, drag_race_data_base).sort_values(by=['season_number'])

#winners
main_season_winners_df = pd.read_sql(main_season_winners_query, drag_race_data_base).astype({'season_number':'int32'}).sort_values(by=['season_number'])
all_stars_winners_df = pd.read_sql(all_stars_winners_query, drag_race_data_base).sort_values(by=['season_number'])

<h2 style="color:deeppink">Sandbox</h2>
<b style="color:deeppink">Use the space below to experiment. Nothing below will be included in the finished product.</b>

In [9]:
main_seasons_df

,name,season_number,place,winner,miss_congeniality,image_url,quote
id,,,,,,,
1,Victoria 'Porkchop' Parker,1,9.0,0,0,http://www.nokeynoshade.party/images/victoria-...,Would you fuck me? I'd fuck me.
2,Tammie Brown,1,8.0,0,0,http://www.nokeynoshade.party/images/tammie-br...,"I'm not a slut, I'm a lady. You look under my ..."
3,Akashia,1,7.0,0,0,http://www.nokeynoshade.party/images/akashia.jpg,"If I was a girl, I'd be a stripper, or a slut ..."
4,Jade,1,6.0,0,0,http://www.nokeynoshade.party/images/jade.jpg,"Jade is definitely my alter-ego, basically she..."
5,Ongina,1,5.0,0,0,http://www.nokeynoshade.party/images/ongina.jpg,"If I don't win this, I swear to god I'm gonna ..."
...,...,...,...,...,...,...,...
140,Kahanna Montrese,11,14.0,0,0,https://www.nokeynoshade.party/images/kahanna-...,Rhinestones can be for pretty girls.
139,Soju,11,15.0,0,0,https://www.nokeynoshade.party/images/soju.jpg,"I have a cyst, I'm currently oozing."
152,Yvie oddly,11,1.0,1,0,https://www.nokeynoshade.party/images/yvie-odd...,"Move over ladies, this race just took an odd t..."


<h2 style="color:deeppink">Export to CSV</h2>

In [10]:
# csv_export(DataFrame, file_name)